In [1]:
import pandas as pd
#from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
import joblib
import hopsworks


In [2]:

project = hopsworks.login()

fs = project.get_feature_store()

2025-04-04 16:50:11,169 INFO: Initializing external client
2025-04-04 16:50:11,170 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-04 16:50:11,683 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-04 16:50:12,448 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [3]:
users_fg = fs.get_feature_group(
    name="users",
    version=1
)

events_fg = fs.get_feature_group(
    name="events",
    version=1
)

weather_rank_fg = fs.get_feature_group(
    name="weather_ranking",
    version=1
)

no_weather_rank_fg = fs.get_feature_group(
    name="no_weather_ranking",
    version=1
)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

In [7]:
# Select features
selected_features_customers = users_fg.select_all()

fs.get_or_create_feature_view( 
    name='users',
    query=selected_features_customers,
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/users/version/1


In [9]:
# Select features
selected_features_articles = events_fg.select_all()

fs.get_or_create_feature_view(
    name='events',
    query=selected_features_articles,
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/events/version/1


In [20]:
# Select features
features_weather_ranking = weather_rank_fg.select_all()

feature_view_ranking_weather = fs.get_or_create_feature_view(
    name='weather_ranking',
    query=features_weather_ranking,
    labels=["label"],
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/weather_ranking/version/1


In [21]:
# Select features
features_no_weather_ranking = no_weather_rank_fg.select_all()

feature_view_ranking_no_weather = fs.get_or_create_feature_view(
    name='no_weather_ranking',
    query=features_no_weather_ranking,
    labels=["label"],
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/no_weather_ranking/version/1


---

---

---

---

In [3]:
# Get feature views
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking', version=1)


In [4]:

feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking', version=1)

In [5]:
# Get training and validation data directly from feature views
weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
    feature_view_ranking_weather.train_test_split(
    test_size=0.1,
    description='Weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.40s) 
2025-04-04 16:50:45,794 WARNING: VersionWarning: Incremented version to `12`.



In [6]:

no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
    feature_view_ranking_no_weather.train_test_split(
    test_size=0.1,
    description='No-weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.04s) 
2025-04-04 16:50:55,834 WARNING: VersionWarning: Incremented version to `9`.



In [45]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Final version without text fields (title, user_interests)
def train_catboost_without_text_fields(
    train_df, val_df, train_y, val_y
):
    # Drop the text fields if present
    text_columns = ["title", "user_interests"]
    train_df = train_df.drop(columns=[col for col in text_columns if col in train_df.columns])
    val_df = val_df.drop(columns=[col for col in text_columns if col in val_df.columns])

    # Identify categorical features
    cat_features = train_df.select_dtypes(include=["object", "bool"]).columns.tolist()

    # Create CatBoost Pools
    train_pool = Pool(train_df, train_y, cat_features=cat_features)
    val_pool = Pool(val_df, val_y, cat_features=cat_features)

    # Train the model
    model = CatBoostClassifier(
        learning_rate=0.2,
        iterations=100,
        depth=10,
        early_stopping_rounds=5,
        use_best_model=True,
        scale_pos_weight=None, 
        verbose=False
    )


    model.fit(train_pool, eval_set=val_pool)

    # Evaluation
    preds = model.predict(val_pool)
    precision, recall, fscore, _ = precision_recall_fscore_support(val_y, preds, average="binary")
    print("\nClassification Report:")
    print(classification_report(val_y, preds))

    metrics = {
        "precision": precision,
        "recall": recall,
        "fscore": fscore,
    }
    from sklearn.metrics import confusion_matrix
    import numpy as np

    preds = model.scores = model.predict_proba(val_pool)[:, 1] 
    print("Predicted Class Distribution:", np.unique(preds, return_counts=True))

    # print("\nConfusion Matrix:")
    # print(confusion_matrix(val_y, preds))

    return model, metrics, val_pool

"✅ Updated CatBoost training function to exclude title and user_interests for memory safety."


'✅ Updated CatBoost training function to exclude title and user_interests for memory safety.'

In [46]:
# Use this function to train on your weather / no-weather datasets
weather_model, weather_metrics, pool_weather_val = train_catboost_without_text_fields(
    train_df=weather_X_train,
    val_df=weather_X_val,
    train_y=weather_y_train,
    val_y=weather_y_val
)



Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.82      0.75      8984
           1       0.72      0.57      0.64      7381

    accuracy                           0.71     16365
   macro avg       0.71      0.69      0.70     16365
weighted avg       0.71      0.71      0.70     16365

Predicted Class Distribution: (array([0.26036305, 0.26521069, 0.27259941, ..., 0.818147  , 0.81962099,
       0.82370505]), array([1, 1, 1, ..., 1, 1, 1]))


In [52]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        weather_X_train.columns, 
        weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 93.30345517167724,
 'user_city': 0.8238453465734555,
 'title': 0.7587034753891037,
 'user_lat': 0.7423353611731195,
 'event_city': 0.7005351182226973,
 'temperature': 0.6971355597219107,
 'user_weather_preference': 0.5473289693968149,
 'weather_condition': 0.46578401536141395,
 'user_lon': 0.4559328686152446,
 'duration': 0.4353449380475777,
 'attendance_rate': 0.36774408358212907,
 'event_indoor_capability': 0.3622609880108651,
 'event_type': 0.16095242574964425,
 'event_lat': 0.12506524006888797,
 'event_lon': 0.053576438409897495,
 'user_id': 0.0,
 'event_id': 0.0}

In [55]:
# Use this function to train on your weather / no-weather datasets
no_weather_model, no_weather_metrics, pool_no_weather_val = train_catboost_without_text_fields(
    train_df=no_weather_X_train,
    val_df=no_weather_X_val,
    train_y=no_weather_y_train,
    val_y=no_weather_y_val
)


Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.82      0.75      8902
           1       0.73      0.57      0.64      7463

    accuracy                           0.71     16365
   macro avg       0.71      0.70      0.70     16365
weighted avg       0.71      0.71      0.70     16365

Predicted Class Distribution: (array([0.26571178, 0.27168428, 0.27318173, ..., 0.81963744, 0.82102738,
       0.82311222]), array([1, 1, 1, ..., 1, 1, 1]))


In [51]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        no_weather_X_train.columns, 
        no_weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 95.26200382902547,
 'attendance_rate': 0.6332116184844625,
 'event_city': 0.5870931554516937,
 'duration': 0.5853972344250435,
 'user_lon': 0.512945258662686,
 'event_lon': 0.5098842138086315,
 'title': 0.46051506428962835,
 'user_city': 0.4411795968552092,
 'event_lat': 0.3814438637043754,
 'event_indoor_capability': 0.2803980245606658,
 'user_lat': 0.2098836281374181,
 'event_type': 0.1360445125946886,
 'user_id': 0.0,
 'event_id': 0.0}

In [49]:
from sklearn.metrics import roc_auc_score, average_precision_score, ndcg_score, precision_score, recall_score
import numpy as np


def evaluate_ranking_model_proba(model, val_pool, val_y, k_list=[5, 10]):
    """
    Evaluate a CatBoost ranking model using predicted probabilities, not binary class outputs.
    """

    # Predict class probabilities (not class labels)
    proba = model.predict_proba(val_pool)[:, 1]  # Probability for class 1

    results = {
        "AUC": roc_auc_score(val_y, proba),
        "Average Precision (MAP)": average_precision_score(val_y, proba),
    }

    # Convert to numpy arrays
    true_labels = np.array(val_y)
    predicted_scores = np.array(proba)

    # Sort by predicted score
    sorted_indices = np.argsort(predicted_scores)[::-1]
    sorted_true = true_labels[sorted_indices]

    for k in k_list:
        top_k = sorted_true[:k]
        precision_at_k = np.mean(top_k)
        recall_at_k = np.sum(top_k) / np.sum(true_labels)
        ndcg_at_k = ndcg_score(
            y_true=true_labels.reshape(1, -1),
            y_score=predicted_scores.reshape(1, -1),
            k=k
        )



        results[f"Precision@{k}"] = precision_at_k
        results[f"Recall@{k}"] = recall_at_k
        results[f"NDCG@{k}"] = ndcg_at_k

    return results

"✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K."


'✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K.'

In [50]:

# Evaluate weather-aware model
weather_scores = evaluate_ranking_model_proba(
    model=weather_model,
    val_pool=pool_weather_val,
    val_y=weather_y_val
)

# Evaluate no-weather model
no_weather_scores = evaluate_ranking_model_proba(
    model=no_weather_model,
    val_pool=pool_no_weather_val,
    val_y=no_weather_y_val
)

# Compare results
print("Weather Model Scores:")
for k, v in weather_scores.items():
    print(f"{k}: {v:.4f}")

print("\nNo-Weather Model Scores:")
for k, v in no_weather_scores.items():
    print(f"{k}: {v:.4f}")


Weather Model Scores:
AUC: 0.7011
Average Precision (MAP): 0.6712
Precision@5: 0.8000
Recall@5: 0.0005
NDCG@5: 0.9344
Precision@10: 0.8000
Recall@10: 0.0011
NDCG@10: 0.8546

No-Weather Model Scores:
AUC: 0.7064
Average Precision (MAP): 0.6755
Precision@5: 0.8000
Recall@5: 0.0005
NDCG@5: 0.6608
Precision@10: 0.9000
Recall@10: 0.0012
NDCG@10: 0.7799


In [54]:
# Connect to Hopsworks Model Registry
model_registry = project.get_model_registry()

In [53]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib

def upload_catboost_model_to_hopsworks(
    model, model_name: str, model_description: str,
    model_registry, X_train, y_train, metrics: dict,
    local_path="model.pkl"
):
    # Save the model locally
    joblib.dump(model, local_path)

    # Create input/output schema
    input_schema = Schema(X_train)
    output_schema = Schema(y_train)
    model_schema = ModelSchema(input_schema, output_schema)

    # Sample input for UI representation
    input_example = X_train.sample().to_dict("records")

    # Create and save model in Hopsworks
    hopsworks_model = model_registry.python.create_model(
        name=model_name,
        metrics=metrics,
        model_schema=model_schema,
        input_example=input_example,
        description=model_description,
    )

    hopsworks_model.save(local_path)

    print(f"✅ Model '{model_name}' successfully uploaded to Hopsworks.")
    return hopsworks_model

"✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference."


'✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference.'

In [56]:
upload_catboost_model_to_hopsworks(
    model=weather_model,
    model_name="weather_ranking_model",
    model_description="Ranking model trained with weather features",
    model_registry=project.get_model_registry(),
    X_train=weather_X_train, 
    y_train=weather_y_train,
    metrics=weather_metrics
)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/283520 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/615 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1586 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/weather_ranking_model/1
✅ Model 'weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'weather_ranking_model', version: 1)

In [57]:
upload_catboost_model_to_hopsworks(
    model=no_weather_model,
    model_name="no_weather_ranking_model",
    model_description="Ranking model trained without weather features",
    model_registry=project.get_model_registry(),
    X_train=no_weather_X_train, 
    y_train=no_weather_y_train,
    metrics=no_weather_metrics
)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/308023 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/503 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1348 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/no_weather_ranking_model/1
✅ Model 'no_weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'no_weather_ranking_model', version: 1)